In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()
db = client.openfoodfacts
collection = db.products

In [110]:
proj = {
    'product_name_fr' : 1, 
    'ingredients' : 1,
    '_id' : 1,
}

addFields = { 
    
    'ingredients_id' : "$ingredients.id",
    
}


match = {
    
    'ingredients' : { 
        "$elemMatch" : { 
            "text" : { '$regex' : "tomate*", '$options' : "i" },
        } 
    },
    
    "product_name_fr" : { '$regex' : "sauce*", '$options' : "i" }
    
}



c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    #{ '$addFields' : addFields },
])

ingredients = pd.DataFrame(list(c_f))

In [130]:
ingredients

,_id,ingredients,product_name_fr
0,0000069162176,"[{'id': 'fr:INGREDIENTES', 'ingredients': [{'i...",Quenelles de brochet sauce Nantua
1,00005010,"[{'percent_min': '65.79', 'rank': 1, 'percent'...",Organic tomato sauce bolognese
2,00021036,"[{'vegan': 'ignore', 'vegetarian': 'ignore', '...",Tomato & Gorgonzola pasta sauce
3,00089227,"[{'vegan': 'yes', 'rank': 1, 'vegetarian': 'ye...",Tomato & Sausage Pasta Sauce
4,0011110791474,"[{'id': 'en:water', 'text': 'Water', 'percent_...",Peri peri chili culinary hot sauce
...,...,...,...
2702,93201834,"[{'percent_min': 8.333333333333334, 'vegan': '...",BBQ Sauce
2703,9300681008403,"[{'vegan': 'yes', 'percent_max': 87.5, 'rank':...",Sweet chili sauce
2704,9556041601214,"[{'rank': 1, 'ingredients': [{'id': 'fr:PILCHA...",Ayam Sardines In Tomato Sauce
2705,9556041611121,"[{'percent_max': 100, 'rank': 1, 'vegan': 'yes...",Sauce aigre-douce Ayam™


In [180]:
id_ = []
ing = []
has_sub_ingredients = []
sub_ingredients = []
text = []
percent_min = []
percent_max = []
percent = []
rank = []
product_name_fr = []

for i in range(len(ingredients['ingredients'])) :
    for dict_ in ingredients['ingredients'][i] : 
        
        product_name_fr.append(ingredients['product_name_fr'])
        
        if 'id' in dict_.keys() :
            id_.append(dict_['id'])
        else :
            id_.append(None)
        if 'ingredients' in dict_.keys() :
            ing.append(dict_['ingredients'])
        else :
            ing.append(None)
        if 'has_sub_ingredients' in dict_.keys() :
            has_sub_ingredients.append(dict_['has_sub_ingredients'])
        else :
            has_sub_ingredients.append(None)
        if 'sub_ingredients' in dict_.keys() :
            sub_ingredients.append(dict_['sub_ingredients'])
        else :
            sub_ingredients.append(None)
        if 'text' in dict_.keys() :
            text.append(dict_['text'])
        else :
            text.append(None)
        if 'percent_min' in dict_.keys() :
            percent_min.append(dict_['percent_min'])
        else :
            percent_min.append(None)
        if 'percent_max' in dict_.keys() :
            percent_max.append(dict_['percent_max'])
        else :
            percent_max.append(None)
        if 'percent' in dict_.keys() :
            percent.append(dict_['percent'])
        else :
            percent.append(None)
        if 'rank' in dict_.keys() :
            rank.append(dict_['rank'])
        else :
            rank.append(None)
    

In [181]:
df_ingredients = pd.DataFrame(
    list(zip(id_,  has_sub_ingredients, sub_ingredients, text, percent_min, percent_max, percent, rank, product_name_fr)), 
    columns =['_id', 'has_sub_ingredients', 'sub_ingredients', 'text', 'percent_min', 'percent_max', 'percent', 'rank', 'product_name_fr'])

In [182]:
df_ingredients

,_id,has_sub_ingredients,sub_ingredients,text,percent_min,percent_max,percent,rank,product_name_fr
0,fr:INGREDIENTES,yes,None,INGREDIENTES,None,None,None,1.0,0 Quenelles de brochet sauce Nantua 1 ...
1,fr:agua,None,None,agua,None,None,None,2.0,0 Quenelles de brochet sauce Nantua 1 ...
2,fr:harina de tngo tierno carne de Lucio 13 %' ...,None,None,harina de tngo tierno carne de Lucio 13%' acei...,None,None,None,3.0,0 Quenelles de brochet sauce Nantua 1 ...
3,fr:Lactosa y proteina léctea,None,None,Lactosa y proteina léctea,None,None,None,4.0,0 Quenelles de brochet sauce Nantua 1 ...
4,en:sal-tree,None,None,sal,None,None,None,5.0,0 Quenelles de brochet sauce Nantua 1 ...
...,...,...,...,...,...,...,...,...,...
63472,en:water,None,None,EAU,0,1.0625,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...
63473,en:salt,None,None,SEL,0,0.85,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...
63474,en:e150a,None,None,e150A,0,2.83333,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...
63475,en:e150a,None,None,e150C,0,1.41667,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...


In [184]:
def label_ingredient (row):
    if row['_id'][:2] == 'en' :
        return 0
    return 1

In [185]:
df_ingredients['unknown_ingredient'] = df_ingredients.apply(lambda row: label_ingredient(row), axis=1)

In [186]:
df_ingredients

,_id,has_sub_ingredients,sub_ingredients,text,percent_min,percent_max,percent,rank,product_name_fr,unknown_ingredient
0,fr:INGREDIENTES,yes,None,INGREDIENTES,None,None,None,1.0,0 Quenelles de brochet sauce Nantua 1 ...,1
1,fr:agua,None,None,agua,None,None,None,2.0,0 Quenelles de brochet sauce Nantua 1 ...,1
2,fr:harina de tngo tierno carne de Lucio 13 %' ...,None,None,harina de tngo tierno carne de Lucio 13%' acei...,None,None,None,3.0,0 Quenelles de brochet sauce Nantua 1 ...,1
3,fr:Lactosa y proteina léctea,None,None,Lactosa y proteina léctea,None,None,None,4.0,0 Quenelles de brochet sauce Nantua 1 ...,1
4,en:sal-tree,None,None,sal,None,None,None,5.0,0 Quenelles de brochet sauce Nantua 1 ...,0
...,...,...,...,...,...,...,...,...,...,...
63472,en:water,None,None,EAU,0,1.0625,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...,0
63473,en:salt,None,None,SEL,0,0.85,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...,0
63474,en:e150a,None,None,e150A,0,2.83333,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...,0
63475,en:e150a,None,None,e150C,0,1.41667,None,NaN,0 Quenelles de brochet sauce Nantua 1 ...,0


In [187]:
df_ingredients.to_csv('O_ingredients.csv')

In [189]:
df_ingredients.iloc[102, 0:-1]

_id                                           en:armidon de maïs modifié
has_sub_ingredients                                                 None
sub_ingredients                                                     None
text                                             armidon de maïs modifié
percent_min                                                            0
percent_max                                                      14.2857
percent                                                             None
rank                                                                   7
product_name_fr        0        Quenelles de brochet sauce Nantua
1  ...
Name: 102, dtype: object

In [188]:
df_ingredients.loc[100, :]

_id                                                               en:eau
has_sub_ingredients                                                 None
sub_ingredients                                                     None
text                                                                 eau
percent_min                                                            0
percent_max                                                           20
percent                                                             None
rank                                                                   5
product_name_fr        0        Quenelles de brochet sauce Nantua
1  ...
unknown_ingredient                                                     0
Name: 100, dtype: object